## Imports

In [1]:
import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))
from bbtoolkit.preprocessing.neural_generators import MTLGenerator,\
    HDGenerator,\
    TCGenerator,\
    GCGenerator, GCMap,\
    PCGenerator
import configparser
from bbtoolkit.data.configparser import EvalConfigParser
import numpy as np
from bbtoolkit.preprocessing.environment import Geometry

In [2]:
mtl_cfg = '../cfg/cells/mtl_cells.ini'
config = EvalConfigParser(interpolation=configparser.ExtendedInterpolation(), allow_no_value=True)
config.read(mtl_cfg)

space_section = config['Space']
res = space_section.eval('res')
r_max = space_section.eval('r_max')

polargrid_section = config['PolarGrid']
polar_dist_res = polargrid_section.eval('polar_dist_res')
polar_ang_res = polargrid_section.eval('polar_ang_res', locals=dict(pi=np.pi))
h_sig = polargrid_section.eval('sigma_hill')

geometry = Geometry.load('../data/envs/squared_room.pkl')

weights = MTLGenerator(
    res, r_max, h_sig, polar_dist_res, polar_ang_res, geometry
).generate()

In [3]:
weights.connection_map

,h,pr,bvc
h,1,1,1
pr,1,1,1
bvc,1,1,1


In [4]:
weights.save('../data/weights/mtl_weights.pkl')

## Head Direction Weights

In [5]:
config_path = '../cfg/cells/hd_cells.ini'
config = EvalConfigParser(interpolation=configparser.ExtendedInterpolation(), allow_no_value=True)
config.read(config_path)

neurons_cfg = config['Neurons']
n_hd = neurons_cfg.eval('n_neurons')
amp_max = neurons_cfg.eval('max_amplitude')
sig_scaled = neurons_cfg.eval('sigma', locals={'pi': np.pi})

training_cfg = config['Training']
n_steps = training_cfg.eval('n_steps')
dt = training_cfg.eval('dt')
log_size = training_cfg.eval('log_size')
decay = training_cfg.eval('decay')

weights = HDGenerator(
    n_hd,
    amp_max,
    sig_scaled,
    n_steps,
    dt,
    log_size,
    decay
).generate()

In [6]:
weights.connection_map

,hd,rot
hd,1,0
rot,0,1


In [7]:
weights.save('../data/weights/hd_weights.pkl')

## Transformation circuit

In [8]:
hd_config_path = '../cfg/cells/hd_cells.ini'
hd_config = EvalConfigParser(interpolation=configparser.ExtendedInterpolation(), allow_no_value=True)
hd_config.read(hd_config_path)

mtl_config_path = '../cfg/cells/mtl_cells.ini'
mtl_config = EvalConfigParser(interpolation=configparser.ExtendedInterpolation(), allow_no_value=True)
mtl_config.read(mtl_config_path)

tr_config_path = '../cfg/cells/transformation_circuit.ini'
tr_config = EvalConfigParser(interpolation=configparser.ExtendedInterpolation(), allow_no_value=True)
tr_config.read(tr_config_path)

r_max = mtl_config['Space'].eval('r_max')

mtl_grid_cfg = mtl_config['PolarGrid']
n_radial_points = mtl_grid_cfg.eval('n_radial_points')
polar_dist_res = mtl_grid_cfg.eval('polar_dist_res')
polar_ang_res = mtl_grid_cfg.eval('polar_ang_res', locals={'pi': np.pi})
h_sig = mtl_grid_cfg.eval('sigma_hill')

tr_space_cfg = tr_config['Space']
tr_res = tr_space_cfg.eval('tr_res', locals={'pi': np.pi})
res = tr_space_cfg.eval('res')

n_steps = tr_config['Training'].eval('n_steps')

hd_neurons_cfg = hd_config['Neurons']
sigma_angular = hd_neurons_cfg.eval('sigma', locals={'pi': np.pi})
n_hd = hd_neurons_cfg.eval('n_neurons')

weights = TCGenerator(
    n_hd,
    tr_res,
    res,
    r_max,
    polar_dist_res,
    n_radial_points,
    polar_ang_res,
    h_sig,
    sigma_angular,
    n_steps
).generate()

In [18]:
weights.connection_map

,tr,pw,bvc,hd
tr,0,1,1,0
bvc,1,0,0,0
hd,1,0,0,0
pw,1,0,0,0


In [10]:
weights.save('../data/weights/tc_weights.pkl')

## Grid Cells Frequency Map

In [11]:
cfg_path = '../cfg/cells/grid_cells.ini'
config = EvalConfigParser()
config.read(cfg_path)

space_config = config['Space']
n_mod = space_config.eval('n_mod')
n_per_mod = space_config.eval('n_per_mod')
res = space_config.eval('res')
x_max = space_config.eval('x_max')
y_max = space_config.eval('y_max')

neurons_config = config['Neurons']
f_mods = neurons_config.eval('f_mods', locals={'pi': np.pi})
fac = neurons_config.eval('FAC')
r_size = neurons_config.eval('r_size')
orientations = neurons_config.eval('orientations', locals={'pi': np.pi})


frmap = GCGenerator(
    res,
    x_max,
    y_max,
    n_mod,
    n_per_mod,
    f_mods,
    fac,
    r_size,
    orientations
).generate()

In [12]:
frmap.save('../data/weights/gc_map.pkl')

## Place Cells

In [13]:
gc_cfg_path = '../cfg/cells/grid_cells.ini'
gc_config = EvalConfigParser()
gc_config.read(gc_cfg_path)

pc_cfg_path = '../cfg/cells/place_cells.ini'
pc_config = EvalConfigParser()
pc_config.read(pc_cfg_path)

gc_space_config = gc_config['Space']
n_mod = gc_space_config.eval('n_mod')
n_per_mod = gc_space_config.eval('n_per_mod')

pc_space_config = pc_config['Space']
res = pc_space_config.eval('res')
x_max = pc_space_config.eval('x_max')
y_max = pc_space_config.eval('y_max')

weights = PCGenerator(
    res=0.5,
    x_max=22,
    y_max=22,
    n_mod=7,
    n_per_mod=100,
    gc_map=GCMap.load('../data/weights/gc_map.pkl')
).generate()

In [14]:
weights.connection_map

,gc,pc
gc,0,1


In [15]:
weights.save('../data/weights/pc_weights.pkl')